In [1]:
#DATA ENGINEERING

In [2]:
import csv
import pandas as pd

# SQL Alchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, Numeric, Text

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

import numpy as np
import os


In [3]:
# Path of the CSV file
csvfile = "hawaii_a_measurements.csv"
csv2 = "hawaii_a_stations.csv"

In [4]:
# Read CSV file into a pandas DataFrame
df = pd.read_csv(csvfile, dtype=object)
df2 = pd.read_csv(csv2, dtype=object)

In [5]:
# Preview DataFrame

df.head()

,obs,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65
1,2,USC00519397,2010-01-02,0,63
2,3,USC00519397,2010-01-03,0,74
3,4,USC00519397,2010-01-04,0,76
4,5,USC00519397,2010-01-06,NaN,73


In [6]:
# Use `dropna` to drop any rows where there is missing data
df = df.dropna(axis=0)
df.head()

,obs,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65
1,2,USC00519397,2010-01-02,0,63
2,3,USC00519397,2010-01-03,0,74
3,4,USC00519397,2010-01-04,0,76
5,6,USC00519397,2010-01-07,0.06,70


In [7]:
# Resetting the index, the gap is gone
df = df.reset_index(drop=True)
df.head()

,obs,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65
1,2,USC00519397,2010-01-02,0,63
2,3,USC00519397,2010-01-03,0,74
3,4,USC00519397,2010-01-04,0,76
4,6,USC00519397,2010-01-07,0.06,70


In [8]:
# Save the cleaned data to a file called `clean_measurement.csv`
cleaned_csv = "cleaned_measurements.csv"
df.to_csv(cleaned_csv, index=False)

In [9]:
# Preview DataFrame

df2.head()

,st_id,station,name,latitude,longitude,elevation
0,1,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3
1,2,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,3,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7
3,4,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,5,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [10]:
#DATABASE ENGINEERING

In [11]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base

from sqlalchemy import Column, Integer, String, Numeric, Text, Float


In [12]:
# Create an engine to a SQLite database file called `clean_measurement.sqlite`
engine = create_engine("sqlite:///cleaned_measurements.sqlite")

In [13]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [14]:
Base = declarative_base()

class Measurement (Base):
    __tablename__ = 'measurement'
    obs = Column (Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)
    
    def __repr__(self):
        return f"station={self.station}, date={date.self}, prcp={prcp.self}, tobs={tobs.self}"
    
class Station (Base):
    __tablename__ = 'station'
    st_id = Column (Integer, primary_key=True) 
    station = Column(String)
    name = Column(String(255))
    latitude =  Column(Float)
    longitude = Column(Float)
    elevation = Column(Float) 
        

In [15]:
# Use `create_all` to create the measurement table in the database
Base.metadata.create_all(engine)

In [16]:
# Load the cleaned csv file into a pandas dataframe
new_df = pd.read_csv(cleaned_csv)

In [17]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
data = new_df.to_dict(orient='records')

In [18]:
# Data is just a list of dictionaries that represent each row of data
print(data[:5])

[{'obs': 1, 'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'obs': 2, 'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'obs': 3, 'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'obs': 4, 'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'obs': 6, 'station': 'USC00519397', 'date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}]


In [19]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [20]:
# Save the reference to the `measurement` table as a variable called `table1`
table1 = sqlalchemy.Table('measurement', metadata, autoload=True)
table1

Table('measurement', MetaData(bind=Engine(sqlite:///cleaned_measurements.sqlite)), Column('obs', INTEGER(), table=<measurement>, primary_key=True, nullable=False), Column('station', VARCHAR(), table=<measurement>), Column('date', VARCHAR(), table=<measurement>), Column('prcp', FLOAT(), table=<measurement>), Column('tobs', INTEGER(), table=<measurement>), schema=None)

In [21]:
#deleting Table1 to remove any pre-existing data.
conn.execute(table1.delete())

In [22]:
conn.execute(table1.insert(), data)

In [23]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (6, 'USC00519397', '2010-01-07', 0.06, 70)]

In [24]:
# Save the reference to the `station` table as a variable called `table1`
table2 = sqlalchemy.Table('station', metadata, autoload=True)
table2

Table('station', MetaData(bind=Engine(sqlite:///cleaned_measurements.sqlite)), Column('st_id', INTEGER(), table=<station>, primary_key=True, nullable=False), Column('station', VARCHAR(), table=<station>), Column('name', VARCHAR(length=255), table=<station>), Column('latitude', INTEGER(), table=<station>), Column('longitude', INTEGER(), table=<station>), Column('elevation', INTEGER(), table=<station>), schema=None)

In [25]:
#deleting Table1 to remove any pre-existing data.
conn.execute(table2.delete())

In [26]:
conn.execute(table2.insert(), data)

In [27]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from station limit 5").fetchall()

[(1, 'USC00519397', None, None, None, None),
 (2, 'USC00519397', None, None, None, None),
 (3, 'USC00519397', None, None, None, None),
 (4, 'USC00519397', None, None, None, None),
 (5, 'USC00519397', None, None, None, None)]